In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from kmodes.kprototypes import KPrototypes
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
RANDOM_STATE = 42

In [30]:
pd.set_option('display.max_columns', None)

In [31]:
def format_lenght(date):
    return date[0]
def K_Prototypes(X):
    kproto_4 = KPrototypes(n_clusters=4, init='Huang', verbose=0, random_state=42, n_jobs=-1)
    clusters = kproto_4.fit_predict(X, categorical=[3, 11, 12, 13, 14, 17, 18, 20, 22])
    return clusters

df = pd.read_csv('train.csv')
df = df.drop(columns=['ID','adr'])
df.dropna(subset=['children'], inplace=True)
df['country']=df['country'].fillna(df['country'].value_counts().index[0]) 
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])
df['hotel'] = np.where(df['hotel']=='City Hotel',1,0)
df['arrival_date_full'] = df['arrival_date_year'].astype(str) + "-" + df['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + df['arrival_date_day_of_month'].astype(str)
df['arrival_date_full'] = pd.to_datetime(df['arrival_date_full'], format="%Y-%m-%d")
df['status_minus_arrival_date'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
X = df.drop(columns=['is_canceled', 'reservation_status', 'agent', 'company', 'country', 'reservation_status_date', 'arrival_date_full'])
clusters = K_Prototypes(X)
df['clusters'] = clusters
df = pd.get_dummies(df, columns=[ 'meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type','country'])
x = df.drop(columns=['is_canceled','arrival_date_year','arrival_date_month','reservation_status', 'agent', 'company', 
                 'reservation_status_date', 'arrival_date_full'])
y = df['is_canceled']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, 
                                                stratify=y, test_size=0.2)

In [32]:
x.head()

,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,status_minus_arrival_date,clusters,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,country_ABW,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,country_AUS,country_AUT,country_AZE,country_BDI,country_BEL,country_BEN,country_BFA,country_BGD,country_BGR,country_BHR,country_BIH,country_BLR,country_BOL,country_BRA,country_BRB,country_BWA,country_CAF,country_CHE,country_CHL,country_CHN,country_CIV,country_CMR,country_CN,country_COL,country_COM,country_CPV,country_CRI,country_CUB,country_CYM,country_CYP,country_CZE,country_DEU,country_DMA,country_DNK,country_DOM,country_DZA,country_ECU,country_EGY,country_ESP,country_EST,country_ETH,country_FIN,country_FRA,country_GAB,country_GBR,country_GEO,country_GGY,country_GHA,country_GIB,country_GLP,country_GNB,country_GRC,country_GTM,country_GUY,country_HKG,country_HND,country_HRV,country_HUN,country_IDN,country_IMN,country_IND,country_IRL,country_IRN,country_IRQ,country_ISL,country_ISR,country_ITA,country_JAM,country_JEY,country_JOR,country_JPN,country_KAZ,country_KEN,country_KHM,country_KNA,country_KOR,country_KWT,country_LBN,country_LBY,country_LCA,country_LIE,country_LKA,country_LTU,country_LUX,country_LVA,country_MAC,country_MAR,country_MCO,country_MDG,country_MDV,country_MEX,country_MKD,country_MLI,country_MLT,country_MMR,country_MNE,country_MOZ,country_MUS,country_MWI,country_MYS,country_NAM,country_NGA,country_NIC,country_NLD,country_NOR,country_NZL,country_OMN,country_PAK,country_PAN,country_PER,country_PHL,country_PLW,country_POL,country_PRI,country_PRT,country_PRY,country_PYF,country_QAT,country_ROU,country_RUS,country_RWA,country_SAU,country_SEN,country_SGP,country_SLV,country_SMR,country_SRB,country_STP,country_SUR,country_SVK,country_SVN,country_SWE,country_SYC,country_SYR,country_TGO,country_THA,country_TJK,country_TMP,country_TUN,country_TUR,country_TWN,country_TZA,country_UGA,country_UKR,country_URY,country_USA,country_UZB,country_VEN,country_VGB,country_VNM,country_ZAF,country_ZMB,country_ZWE
0,0,342,27,1,0,0,2,0.0,0,0,0,0,3,0,0,0,0,3,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,257,27,1,0,2,1,0.0,0,0,0,0,1,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:

df = pd.read_csv('test.csv')
df = df.drop(columns=['ID'])
#df['agent'] = df['agent'].fillna('No_Agent')
#df['company'] = df['company'].fillna('No_Company')
#df['country'].fillna("unknown", inplace=True)  
#df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])
df['country']=df['country'].fillna(df['country'].value_counts().index[0]) 
df['hotel'] = np.where(df['hotel']=='City Hotel',1,0)
df['arrival_date_full'] = df['arrival_date_year'].astype(str) + "-" + df['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + df['arrival_date_day_of_month'].astype(str)
df['arrival_date_full'] = pd.to_datetime(df['arrival_date_full'], format="%Y-%m-%d")
df['status_minus_arrival_date'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
X = df.drop(columns=['agent', 'company', 'country', 'arrival_date_full'])
clusters = K_Prototypes(X)
df['clusters'] = clusters
df = pd.get_dummies(df, columns=['meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type','country'])
test = df.drop(columns=['arrival_date_year','arrival_date_month', 'agent', 'company',  'arrival_date_full'])


In [34]:
def missing_col(x_train,x_pred):
    missing_cols = set( x_train.columns ) - set( x_pred.columns )

    for c in missing_cols:
        x_pred[c] = 0

    x_pred = x_pred[x_train.columns]
    return x_pred

In [35]:
x_pred = missing_col(x_train, test)

In [36]:
def DTree(x_train, x_test, y_train, y_test):
    dt_params = {
    'max_depth':[None],
    'max_features' : [0.7],
    'min_samples_split': [25], 
    'min_samples_leaf': [1]
    }

    # Perform Grid Search
    dt_gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                     dt_params,
                     cv = 5,
                     scoring = 'accuracy')
    dt = dt_gs.fit(x_train, y_train)
    
    print(f'Best Training Accuracy: {dt.score(x_train, y_train)}')
    print(f'Best Testing Accuracy: {dt.score(x_test, y_test)}')
    return dt


In [37]:
clf = DTree(x_train, x_test, y_train, y_test)
y_pred_cancel = clf.predict(x_pred)
print(sum(y_pred_cancel == 1))
print(sum(y_pred_cancel != 1))

Best Training Accuracy: 0.9206238647382582
Best Testing Accuracy: 0.8617939473396701
8764
19095


In [5]:
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [6]:
adr_df = pd.read_csv('train.csv')
adr_df = adr_df.drop(columns=['ID','is_canceled','arrival_date_year','arrival_date_month','reservation_status', 'reservation_status_date'])
adr_df.dropna(subset=['children'], inplace=True)
adr_df['country']=adr_df['country'].fillna(adr_df['country'].value_counts().index[0])
adr_df['agent']=adr_df['agent'].fillna(adr_df['agent'].value_counts().index[0])
adr_df['company']=adr_df['company'].fillna(0)
adr_df['children']=adr_df['children'].fillna(adr_df['children'].median())
adr_df['children'] = adr_df['children'].astype(int)   
adr_df['hotel'] = np.where(adr_df['hotel']=='City Hotel',1,0)
adr_df['status_minus_arrival_date'] = adr_df['stays_in_week_nights'] + adr_df['stays_in_weekend_nights']
# X = df.drop(columns=['country','adr'])
# clusters = K_Prototypes(X)
# df['clusters'] = clusters
adr_df.agent = adr_df.agent.astype("category").cat.codes
adr_df.company = adr_df.company.astype("category").cat.codes
adr_df.country = adr_df.country.astype("category").cat.codes
adr_df.meal = adr_df.meal.astype("category").cat.codes
adr_df.market_segment = adr_df.market_segment.astype("category").cat.codes
adr_df.distribution_channel = adr_df.distribution_channel.astype("category").cat.codes
adr_df.reserved_room_type = adr_df.reserved_room_type.astype("category").cat.codes
adr_df.assigned_room_type = adr_df.assigned_room_type.astype("category").cat.codes
adr_df.deposit_type = adr_df.deposit_type.astype("category").cat.codes
adr_df.customer_type = adr_df.customer_type.astype("category").cat.codes
x_adr = adr_df.drop(columns=['adr'])
y_adr = adr_df['adr']
x_train_adr, x_val_adr, y_train_adr, y_val_adr = train_test_split(x_adr, y_adr, random_state=42)

In [8]:
ytrain = np.reshape(y_train_adr.values,(-1,1))
yval = np.reshape(y_val_adr.values,(-1,1))
scaler_x = StandardScaler()
scaler_y = StandardScaler()

xtrain_scale= scaler_x.fit_transform(x_train_adr)
xval_scale=scaler_x.transform(x_val_adr)

ytrain_scale=scaler_y.fit_transform(ytrain)
yval_scale=scaler_y.transform(yval)


In [17]:
svr = SVR(C=6)
m = svr.fit(xtrain_scale, ytrain_scale.ravel())
print(f'Best Training Accuracy: {m.score(xtrain_scale, ytrain_scale)}')
print(f'Best Testing Accuracy: {m.score(xval_scale, yval_scale)}')

Best Training Accuracy: 0.6667438596809718
Best Testing Accuracy: 0.7125903386665171


In [20]:
adr_test = pd.read_csv('test.csv')
adr_test = adr_test.drop(columns=['ID','arrival_date_year','arrival_date_month'])
adr_test.dropna(subset=['children'], inplace=True)
adr_test['country']=adr_test['country'].fillna(adr_test['country'].value_counts().index[0])
adr_test['agent']=adr_test['agent'].fillna(adr_test['agent'].value_counts().index[0])
adr_test['company']=adr_test['company'].fillna(0)
adr_test['children']=adr_test['children'].fillna(adr_test['children'].median())
adr_test['children'] = adr_test['children'].astype(int)   
adr_test['hotel'] = np.where(adr_test['hotel']=='City Hotel',1,0)
adr_test['status_minus_arrival_date'] = adr_test['stays_in_week_nights'] + adr_test['stays_in_weekend_nights']
adr_test.agent = adr_test.agent.astype("category").cat.codes
adr_test.company = adr_test.company.astype("category").cat.codes
adr_test.country = adr_test.country.astype("category").cat.codes
adr_test.meal = adr_test.meal.astype("category").cat.codes
adr_test.market_segment = adr_test.market_segment.astype("category").cat.codes
adr_test.distribution_channel = adr_test.distribution_channel.astype("category").cat.codes
adr_test.reserved_room_type = adr_test.reserved_room_type.astype("category").cat.codes
adr_test.assigned_room_type = adr_test.assigned_room_type.astype("category").cat.codes
adr_test.deposit_type = adr_test.deposit_type.astype("category").cat.codes
adr_test.customer_type = adr_test.customer_type.astype("category").cat.codes


In [25]:
x_train_adr.columns

Index(['hotel', 'lead_time', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests',
       'status_minus_arrival_date'],
      dtype='object')

In [24]:
adr_test.columns

Index(['hotel', 'lead_time', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests',
       'status_minus_arrival_date'],
      dtype='object')

In [26]:
adr_test = scaler_x.transform(adr_test)
adr_test

array([[ 0.7202746 , -0.20334735, -1.01029741, ..., -0.2629512 ,
         0.61678056,  1.41876756],
       [ 0.7202746 ,  1.05881274, -1.01029741, ..., -0.2629512 ,
         0.61678056,  4.13513012],
       [-1.38835939, -0.80121265, -1.01029741, ..., -0.2629512 ,
         0.61678056,  1.41876756],
       ...,
       [-1.38835939, -0.75376303,  0.45335969, ..., -0.2629512 ,
         1.92128208, -0.13343962],
       [-1.38835939,  0.897484  ,  0.45335969, ..., -0.2629512 ,
        -0.68772096,  1.41876756],
       [ 0.7202746 , -0.88662198,  0.45335969, ..., -0.2629512 ,
         4.53028511, -0.90954321]])

In [27]:
adr_pred = m.predict(adr_test)

In [113]:
fin_adr = scaler_y.inverse_transform(adr_pred)

final = pd.DataFrame(fin_adr , columns=['adr'])
final['cancel'] = y_pred_cancel
final['cancel'] = np.where(final['cancel']==0,1,0)


In [1]:
y_pred_cancel
cancel = pd.DataFrame(y_pred_cancel, columns='Dtree')
cancel.to_csv('cancelled',index=False)

NameError: name 'y_pred_cancel' is not defined

In [115]:
df = pd.read_csv('test.csv')
df['arrival_date_full'] = df['arrival_date_year'].astype(str) + "-" + df['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + df['arrival_date_day_of_month'].astype(str)
df['arrival_date_full'] = pd.to_datetime(df['arrival_date_full'], format="%Y-%m-%d")
df['status_minus_arrival_date'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

In [116]:
final['arrival_date_full'] = df['arrival_date_full']
final['status_minus_arrival_date'] = df['status_minus_arrival_date'].astype(int)
final.arrival_date_full

0       2017-04-01
1       2017-04-01
2       2017-04-01
3       2017-04-01
4       2017-04-01
           ...    
27854   2017-08-31
27855   2017-08-31
27856   2017-08-31
27857   2017-08-31
27858   2017-08-31
Name: arrival_date_full, Length: 27859, dtype: datetime64[ns]

In [117]:
final['score'] = (final['status_minus_arrival_date'] * final['cancel'] * final['adr'])
final = final.groupby(final.arrival_date_full).sum().astype(int)
final['score'] = ((final['score'])/10000).astype(int)

In [5]:
adr = pd.read_csv('adr.csv')
final['score']

NameError: name 'final' is not defined

In [118]:
final.score.values

array([ 4,  3,  5,  2,  4,  5,  3,  7,  9,  5,  3,  3,  7,  6,  7,  6,  5,
        2,  3,  4,  6,  6,  5,  4,  2,  5,  4,  7,  9,  4,  6,  4,  5,  6,
        6,  7,  5,  6,  4,  4,  5,  4,  6,  6,  9,  4,  4,  5,  7,  7,  5,
        5,  3,  4,  6,  4,  4,  8,  4,  4,  4,  6,  7,  6,  5,  6,  3,  3,
        9,  6,  5,  8,  6,  3,  4,  6,  5, 10,  5,  7,  5,  5,  5,  5,  6,
        6,  7,  7,  5,  5,  5,  8,  6,  9,  6,  4,  6,  5,  6,  8,  7,  6,
        6,  5,  6, 10,  7,  9,  5,  3,  5,  5,  8,  8,  9,  5,  6,  6,  5,
        9,  6,  8,  7,  5,  7,  5,  7,  8, 10,  5,  5,  6,  5,  8,  7,  7,
        5,  7,  8,  7,  8,  8,  7,  3,  5,  5,  5,  8,  6,  8,  4,  3,  5])

In [120]:
ans = pd.read_csv('test_nolabel.csv')
ans['label'] = final.score.values
ans.to_csv('final_ans1.csv',index=False)